In [4]:
import pandas as pd
import pyodbc
from sqlpush.sqlstd import *
from sqlpush.config import *
from tabulate import tabulate

In [5]:
# how to fuzzy match : 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [6]:
# Prepare connection
cursor, conn = connectdb(config)

In [145]:
# let's define all the needed functions

def getdata(cursor):
    query = "SELECT * FROM belgium_addresses "
    cursor.execute(query)
    tablecol = [i[0] for i in cursor.fetchall()]
    return tablecol


def get_street_id_from_postcode(cursor, postcode):
    query = "SELECT street_id FROM belgium_streets WHERE postcode = '%s'" %(postcode)
    cursor.execute(query)
    tablecol = [i[0] for i in cursor.fetchall()]
    return tablecol

def get_language(streetname):
    streetname = streetname.lower()
    language = "None"
    if "straat" in streetname :
        language = "NL"
    elif "laan" in streetname :
        language = "NL"
    elif "rue" in streetname :
        language = "FR"
    elif "avenue" in streetname :
        language = "FR"
    elif "boulevard" in streetname :
        language = "FR"
    elif "square " in streetname :
        language = "FR"
    elif "allées" in streetname :
        language = "FR"
    else : 
        language = "None"
    return language

def get_street_names_from_postcode_lg(cursor, postcode, language):
    if language == "FR" :
        query = "SELECT streetname_fr FROM belgium_streets WHERE postcode = '%s'" %(postcode)
        cursor.execute(query)
        tablecol = [i[0] for i in cursor.fetchall()]
    elif language == "NL" :
        query = "SELECT streetname_nl FROM belgium_streets WHERE postcode = '%s'" %(postcode)
        cursor.execute(query)
        tablecol = [i[0] for i in cursor.fetchall()]
    else : 
        query = "SELECT streetname_fr, streetname_nl FROM belgium_streets WHERE postcode = '%s'" %(postcode)
        cursor.execute(query)
        tablecol = cursor.fetchall()
    return tablecol

# I need a function that will take a list of names as input and a search term
# will build another list with fuzzy match score, and return the value with the highest score
def get_best_matching(list_of_streets, term) :
    scores = []
    for element in list_of_streets : 
        scores.append(fuzz.token_sort_ratio(element, term))
    return scores

# takes as input a 2D list, outupts a 1D list
def transform_list(street_n_list) : 
    total_list = []
    for element in street_n_list : 
        total_list.append(element[0])
        total_list.append(element[1])

def get_best_address_given_scores(scores, list_of_streets):
    max_value = max(scores) 
    max_index = scores. index(max_value)
    return list_of_streets[max_index]

def get_street_ids_give_name(street_name, language) :
    if language == "FR" :
        query = "SELECT street_id FROM belgium_streets WHERE streetname_fr = '%s'" %(street_name)
        cursor.execute(query)
        tablecol = [i[0] for i in cursor.fetchall()]
        return tablecol
    elif language == "NL" : 
        query = "SELECT street_id FROM belgium_streets WHERE streetname_nl = '%s'" %(street_name)
        cursor.execute(query)
        tablecol = [i[0] for i in cursor.fetchall()]
        return tablecol
    else :  
        query = "SELECT street_id FROM belgium_streets WHERE streetname_nl = '%s' OR streetname_fr = '%s'" %(street_name)
        cursor.execute(query)
        tablecol = [i[0] for i in cursor.fetchall()]
        return tablecol

def get_address_info(street_id, house_number) : 
    query = "SELECT * FROM belgium_addresses WHERE street_id = '%s' AND house_number = '%s' AND status <> 'retired'" %(street_id, house_number)
    cursor.execute(query)
    tablecol = cursor.fetchall()
    return tablecol

def get_address_info_wou_housenb(street_id) : 
    query = "SELECT * FROM belgium_addresses WHERE street_id = '%s' AND status <> 'retired'" %(street_id)
    cursor.execute(query)
    tablecol = cursor.fetchall()
    return tablecol

def get_flood_risk_given_addressID(address_ID) : 
    query = "SELECT * FROM flooding_risk WHERE address_id = '%s'" %(address_ID)
    cursor.execute(query)
    tablecol = cursor.fetchall()
    return tablecol

# build a function that given the street name, it outputs a list of all the house numbers, sorted!
def get_house_numbers(street_id) :
    query = "SELECT house_number FROM belgium_addresses WHERE street_id = '%s'" %(street_id)
    cursor.execute(query)
    tablecol = [i[0] for i in cursor.fetchall()]
    int_tablecol = []
    for element in tablecol : 
        try : 
            element = int(element)
            int_tablecol.append(element)
        except : 
            continue
    int_tablecol = list(dict.fromkeys(int_tablecol))
    int_tablecol.sort()
    return int_tablecol

# given a list of house numbers and a house number, check if the house number can be in it, in a logical sense
def is_house_number_feasible(house_number, house_numbers) :
    max_house_num = house_numbers[len(house_numbers) - 1]
    house_number = int(house_number)
    max_house_num = int(max_house_num)
    feasible = True
    if house_number > max_house_num + 20 or house_number < house_numbers[0] - 10 or house_number < 0 : 
        feasible = False
    return feasible

# given a list of house numbers and a house number, give me the values of the house numbers between the given house number (this phrase doesn't make sense at all)
def get_upper_lower_house_numbers(house_number, house_numbers) : 
    # insert element in the list, sort the list, get element position, then get previous and future elements
    house_number = int(house_number)
    house_numbers.append(int(house_number))
    house_numbers.sort()
    index = house_numbers.index(house_number)
    prev = 0
    next = 0
    if index == 0 : 
        next = house_numbers[index + 1]
    elif index == len(house_numbers)-1 :
        prev = house_numbers[index - 1]
    else : 
        next = house_numbers[index + 1] 
        prev = house_numbers[index - 1]
    return [prev, next]


def return_flood_factors(postcode, streetname, house_number) :

    ##################

    best_street_match = ""
    language = get_language(streetname)
    if language == "None" : 
        sn_none = get_street_names_from_postcode_lg(cursor, postcode, language)
        sn_none = transform_list(sn_none)
        scores = get_best_matching(sn_none, streetname)
        best_street_match = get_best_address_given_scores(scores, sn_none)
    else : 
        sn = get_street_names_from_postcode_lg(cursor, postcode, language)
        scores = get_best_matching(sn, streetname)
        best_street_match = get_best_address_given_scores(scores, sn)
    street_ids = get_street_ids_give_name(best_street_match, language)

    #################

    info = get_address_info(street_ids[0], house_number)    
    if len(info) > 0 : 
        # best case
        coordinates_x = info[0][1]
        coordinates_y = info[0][2]
        print("Here are the coordinates : ")
        print("X Coordinates : ", coordinates_x)
        print("Y Coordinates : ", coordinates_y)
        flood_data = get_flood_risk_given_addressID(info[0][0])
        if len(info) > 1 : 
            col_names = ["address id", "coord X", "coord Y", "lat", "long", "box_number", "housenumber", "municipality_id", "municipality_name_de", "municipality_name_fr", "municipality_name_nl", "postcode", "postname_fr", "postname_nl", "street_id", "streetname_de", "streetname_fr", "streetname_nl", "region_code", "status"]
            print(tabulate([info[0]], headers=col_names, tablefmt="fancy_grid"))
            print("Her are the flooding data for this particular address : " + streetname + " " + house_number + " " + postcode)
            col_names = ["address_id", "risk_level", "alea_type", "map_object_id", "map_local_id", "map_src_file"]
            print(tabulate(flood_data, headers=col_names, tablefmt="fancy_grid"))
        else : 
            col_names = ["address id", "coord X", "coord Y", "lat", "long", "box_number", "housenumber", "municipality_id", "municipality_name_de", "municipality_name_fr", "municipality_name_nl", "postcode", "postname_fr", "postname_nl", "street_id", "streetname_de", "streetname_fr", "streetname_nl", "region_code", "status"]
            print(tabulate(info, headers=col_names, tablefmt="fancy_grid"))
            print("Her are the flooding data for this particular address : " + streetname + " " + house_number + " " + postcode)
            col_names = ["address_id", "risk_level", "alea_type", "map_object_id", "map_local_id", "map_src_file"]
            print(tabulate(flood_data, headers=col_names, tablefmt="fancy_grid"))

    elif len(info) == 0 : 
        # let's implement it here and change the entire structuer of this section
        # let's first get the house numbers on this street
        house_numbers = get_house_numbers(street_ids[0])
        # is the house number feasible?
        is_feasible = is_house_number_feasible(house_number, house_numbers)
        if is_feasible : 
            prev_next = get_upper_lower_house_numbers(house_number, house_numbers)
            prev = prev_next[0]
            next = prev_next[1]
            print("We can't find the address with the given house number : ", house_number)
            print("But the house number is close to the following house numbers found in the database : " + str(prev) + " and " + str(next))
            print("which have the following flooding factor : ")
            # get flooding risk
            address_id_prev = get_address_info(street_ids[0], prev)
            address_id_next = get_address_info(street_ids[0], next)
            address_id_next = address_id_next[0][0]
            address_id_prev = address_id_prev[0][0]
            flood_data = []
            dt_prev = get_flood_risk_given_addressID(address_id_prev)
            dt_next = get_flood_risk_given_addressID(address_id_next)
            flood_data.append(dt_prev[0])
            flood_data.append(dt_next[0])
            col_names = ["address_id", "risk_level", "alea_type", "map_object_id", "map_local_id", "map_src_file"]
            print(tabulate(flood_data, headers=col_names, tablefmt="fancy_grid"))     
        else : 
            print("No data was found given the street id and house number, but here is what we can find without the house number : ")
            info_wo = get_address_info_wou_housenb(street_ids[0])
            info_wo = info_wo[0:5]
            col_names = ["address id", "coord X", "coord Y", "lat", "long", "box_number", "housenumber", "municipality_id", "municipality_name_de", "municipality_name_fr", "municipality_name_nl", "postcode", "postname_fr", "postname_nl", "street_id", "streetname_de", "streetname_fr", "streetname_nl", "region_code", "status"]
            print(tabulate(info_wo, headers=col_names, tablefmt="fancy_grid"))
            print("For each of these addresses, here are the flooding info data : ")
            flood_data = []
            for element in info_wo : 
                dt = get_flood_risk_given_addressID(element[0])
                flood_data.append(dt[0])
            col_names = ["address_id", "risk_level", "alea_type", "map_object_id", "map_local_id", "map_src_file"]
            print(tabulate(flood_data, headers=col_names, tablefmt="fancy_grid"))
        
    else : 
        print("Unusual result, but let's print it out : ")
        col_names = ["address id", "coord X", "coord Y", "lat", "long", "box_number", "housenumber", "municipality_id", "municipality_name_de", "municipality_name_fr", "municipality_name_nl", "postcode", "postname_fr", "postname_nl", "street_id", "streetname_de", "streetname_fr", "streetname_nl", "region_code", "status"]
        print(tabulate(info, headers=col_names, tablefmt="fancy_grid"))



        

In [146]:
# let's implement the algorithm

# take the inputs : postcode, streetname and house number
postcode = input("Ener postcode : ")
streetname = input("Enter streetname : ")
house_number = input("Enter house number : ")

In [151]:
return_flood_factors(postcode="1040", streetname="Rue Louis Hap", house_number="246")

We can't find the address with the given house number :  246
But the house number is close to the following house numbers found in the database : 244 and 248
which have the following flooding factor : 
╒══════════════╤══════════════╤═════════════╤═════════════════╤════════════════╤════════════════╕
│   address_id │ risk_level   │ alea_type   │ map_object_id   │ map_local_id   │ map_src_file   │
╞══════════════╪══════════════╪═════════════╪═════════════════╪════════════════╪════════════════╡
│       558218 │ No risk      │             │                 │                │                │
├──────────────┼──────────────┼─────────────┼─────────────────┼────────────────┼────────────────┤
│       515071 │ No risk      │             │                 │                │                │
╘══════════════╧══════════════╧═════════════╧═════════════════╧════════════════╧════════════════╛


In [148]:
return_flood_factors(postcode="1040", streetname="Rue Louisuiana Hap", house_number="236")

Here are the coordinates : 
X Coordinates :  151836.148
Y Coordinates :  169624.559
╒══════════════╤═══════════╤═══════════╤════════╤═════════╤══════════════╤═══════════════╤═══════════════════╤════════════════════════╤════════════════════════╤════════════════════════╤════════════╤════════════════════════════════╤══════════════════════════════╤═════════════╤═════════════════╤═════════════════╤═════════════════╤═══════════════╤══════════╕
│   address id │   coord X │   coord Y │    lat │    long │ box_number   │   housenumber │   municipality_id │ municipality_name_de   │ municipality_name_fr   │ municipality_name_nl   │   postcode │ postname_fr                    │ postname_nl                  │   street_id │ streetname_de   │ streetname_fr   │ streetname_nl   │ region_code   │ status   │
╞══════════════╪═══════════╪═══════════╪════════╪═════════╪══════════════╪═══════════════╪═══════════════════╪════════════════════════╪════════════════════════╪════════════════════════╪════════════╪══

In [149]:
return_flood_factors(postcode="1080", streetname="Boulevard Louis Mettewie", house_number="62")

Here are the coordinates : 
X Coordinates :  145755.97
Y Coordinates :  171729.96
╒══════════════╤═══════════╤═══════════╤═════════╤═════════╤══════════════╤═══════════════╤═══════════════════╤════════════════════════╤════════════════════════╤════════════════════════╤════════════╤══════════════════════╤═════════════════════╤═════════════╤═════════════════╤══════════════════════════╤════════════════════╤═══════════════╤══════════╕
│   address id │   coord X │   coord Y │     lat │    long │ box_number   │   housenumber │   municipality_id │ municipality_name_de   │ municipality_name_fr   │ municipality_name_nl   │   postcode │ postname_fr          │ postname_nl         │   street_id │ streetname_de   │ streetname_fr            │ streetname_nl      │ region_code   │ status   │
╞══════════════╪═══════════╪═══════════╪═════════╪═════════╪══════════════╪═══════════════╪═══════════════════╪════════════════════════╪════════════════════════╪════════════════════════╪════════════╪═══════════════

In [150]:
return_flood_factors(postcode="9500", streetname="Gaverstraat", house_number="2")

Here are the coordinates : 
X Coordinates :  88086.0
Y Coordinates :  168024.0
╒══════════════╤═══════════╤═══════════╤═════════╤════════╤══════════════╤═══════════════╤═══════════════════╤════════════════════════╤════════════════════════╤════════════════════════╤════════════╤═══════════════╤═══════════════╤═════════════╤═════════════════╤═════════════════╤═════════════════╤═══════════════╤══════════╕
│   address id │   coord X │   coord Y │     lat │   long │ box_number   │   housenumber │   municipality_id │ municipality_name_de   │ municipality_name_fr   │ municipality_name_nl   │   postcode │ postname_fr   │ postname_nl   │   street_id │ streetname_de   │ streetname_fr   │ streetname_nl   │ region_code   │ status   │
╞══════════════╪═══════════╪═══════════╪═════════╪════════╪══════════════╪═══════════════╪═══════════════════╪════════════════════════╪════════════════════════╪════════════════════════╪════════════╪═══════════════╪═══════════════╪═════════════╪═════════════════╪═══════

In [156]:
return_flood_factors(postcode="9500", streetname="Gaverstraat", house_number="3")

We can't find the address with the given house number :  3
But the house number is close to the following house numbers found in the database : 2 and 4
which have the following flooding factor : 
╒══════════════╤══════════════╤═════════════╤═════════════════╤════════════════╤═════════════════════════════════════════════════════════════════════════════════════╕
│   address_id │ risk_level   │ alea_type   │   map_object_id │   map_local_id │ map_src_file                                                                        │
╞══════════════╪══════════════╪═════════════╪═════════════════╪════════════════╪═════════════════════════════════════════════════════════════════════════════════════╡
│      1704904 │ potential    │             │          346549 │         346549 │ Overstromingsgevoelige_gebieden_2017_(Watertoets)_correctie_13_07_2017_GewVLA_Shape │
├──────────────┼──────────────┼─────────────┼─────────────────┼────────────────┼────────────────────────────────────────────────────────